# Script para a inserção das entidades PEPs no banco


In [1]:
import pandas as pd
import os, sys, django, neomodel, datetime
import dotenv

BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
sys.path.append(BASE_DIR)
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "tcc.settings")
django.setup()
dotenv.load_dotenv()

from myapi.models import Pessoa, Organizacao, Cargo

### Setup da API da Wikimedia

In [2]:
# url = 'https://en.wikipedia.org/w/api.php'
# search_query = 'AAVA SANTIAGO AGUIAR'
# number_of_results = 1
# parameters = {
#     "action": "query",
#     "list": "search",
#     "srsearch": search_query,
#     "srlimit": 1,
#     "format": "json"
# }

# headers = {
#   'Authorization': f'Bearer {os.getenv('WIKI_TOKEN')}',
#   'User-Agent': f'{os.getenv('WIKI_APP_NAME')}'
# }

# resposta_wiki = requests.get(url, headers=headers, params=parameters)
# data = resposta_wiki.json()
# print(data)

In [3]:
def insere_pep(nome, cpf, nascimento, cnpj) -> Pessoa:
    try:
        pep = Pessoa(nome=nome, 
                     cpf=cpf, 
                     data_nascimento=pd.to_datetime(nascimento, format='%d/%m/%Y', errors='coerce'), 
                     cnpj=cnpj).save()
        print(f'Pessoa {nome} inserida com sucesso.')
        return pep
    except Exception as e:
        print(f"Erro ao inserir PEP: {e}")
    return None

def insere_organizacao(nome, cnpj) -> Organizacao:
    try:
        org = Organizacao(nome=nome, cnpj=cnpj).save()
        print(f'Organização {nome} inserida com sucesso.')
        return org
    except Exception as e:
        print(f"Erro ao inserir organização: {e}")
    return None

def insere_cargo(nome) -> Cargo:
    try:
        cargo = Cargo(nome=nome).save()
        print(f'Cargo {nome} inserido com sucesso.')
        return cargo
    except Exception as e:
        print(f"Erro ao inserir cargo: {e}")
    return None

def relaciona_cargo_pessoa_organizacao(cargo: Cargo, pessoa: Pessoa, org:Organizacao, papel, inicio, fim) -> None:
    try:
        pessoa.cargo.connect(cargo, {'papel': papel,
                                    'data_inicio': pd.to_datetime(inicio, format='%d/%m/%Y', errors='coerce'), 
                                    'data_fim': pd.to_datetime(fim, format='%d/%m/%Y', errors='coerce') if not pd.isna(fim) else None,
                                    'grau_precisao': 5})
        cargo.pertence.connect(org)
        print(f'Relação entre {pessoa.nome}, {cargo.nome} e {org.nome} criada com sucesso.')
    except Exception as e:
        print(f"Erro ao criar relação: {e}")
        return

In [ ]:
csv_path = os.path.join(BASE_DIR, 'pipelines', 'data', '202507_PEP.csv')
if os.path.exists(csv_path):
	df = pd.read_csv(csv_path, encoding='latin1', dtype=str, on_bad_lines='warn', sep=';').fillna('')
else:
	print(f"Arquivo não encontrado: {csv_path}")

CPF                      object
Nome_PEP                 object
Sigla_Função             object
Descrição_Função         object
Nível_Função             object
Nome_Órgão               object
Data_Início_Exercício    object
Data_Fim_Exercício       object
Data_Fim_Carência        object
dtype: object

In [ ]:
for index, row in df.iterrows():
    nome = row['Nome_PEP']
    cpf = row['CPF']
    nascimento = None
    cnpj = None
    org_nome = row['Nome_Órgão']
    org_cnpj = None
    cargo_nome = row['Descrição_Função']
    inicio = row['Data_Início_Exercício']
    fim = row['Data_Fim_Exercício']

    try:
        pessoa = Pessoa.nodes.get(nome=nome, cpf=cpf)
    except neomodel.DoesNotExist as e:
        pessoa = insere_pep(nome, cpf, nascimento, cnpj)

    try:
        org = Organizacao.nodes.get(nome=org_nome)
    except neomodel.DoesNotExist as e:
        org = insere_organizacao(org_nome, org_cnpj)

    try:
        cargo = Cargo.nodes.get(nome=cargo_nome)
    except neomodel.DoesNotExist as e:
        cargo = insere_cargo(cargo_nome)

    if pessoa != None and org != None and cargo != None:
        relaciona_cargo_pessoa_organizacao(cargo, pessoa, org, papel=cargo_nome, inicio=inicio, fim=fim)

Erro ao criar relação: Attempting to inflate property 'data_fim' on <Relationship element_id='5:9ae96d80-38ac-4f87-a4bd-68ab5f89d511:1152921504606846978' nodes=(<Node element_id='4:9ae96d80-38ac-4f87-a4bd-68ab5f89d511:2' labels=frozenset() properties={}>, <Node element_id='4:9ae96d80-38ac-4f87-a4bd-68ab5f89d511:4' labels=frozenset() properties={}>) type='OCUPA' properties={'data_inicio': '2019-12-21T00:00:00', 'grau_precisao': 5, 'data_fim': 'NaT', 'papel': 'DIR. SUPERINTENDENTE'}> of class 'Relacao_Cargo': time data 'Na' does not match format '%Y-%m-%d'
Relação entre AARAO DE MOURA BRITO NETO, PREFEITO e MUN. DE MANGARATIBA-RJ criada com sucesso.
Relação entre AAVA SANTIAGO AGUIAR, VEREADOR e GOIÂNIA-GO criada com sucesso.
Relação entre ABADIA DE MOURA MORAES, VEREADOR e MUN. DE ARAPUTANGA-MT criada com sucesso.
Relação entre ABADIAS SOUZA DO NASCIMENTO DAMASCENO, VEREADOR e ÁGUAS LINDAS DE GOIÁS-GO criada com sucesso.
Relação entre ABADIO RIBEIRO DA ROCHA, VEREADOR e FIGUEIRÃO-MS cri